In [3]:
import pandas as pd

# Load Excel

df_signs = pd.read_excel("Final mental health data .xlsx", sheet_name="Signs")
df_recs = pd.read_excel("Final mental health data .xlsx", sheet_name="Reccomendatons")

grouped = df_signs.groupby('Disorder')['Signs'].apply(lambda signs: ' '.join(signs.astype(str))).reset_index()



In [4]:
# Create the dictionary from the DataFrame
signs_dict = dict(zip(grouped['Disorder'], grouped['Signs']))


In [5]:
from sentence_transformers import SentenceTransformer, util

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode disorder descriptions
disorder_embeddings = {
    disorder: model.encode(sign_text)
    for disorder, sign_text in signs_dict.items()
}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [66]:
# 6. Define prediction function
def predict_disorder_input(user_input, top_k=3):
    print("\n Predicted Disorders & Similarities:\n")

    input_embed = model.encode(user_input)
    similarities = {}

    for disorder, embed in disorder_embeddings.items():
        score = util.cos_sim(input_embed, embed).item()
        similarities[disorder] = score

    # Sort by score
    sorted_disorders = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]

    for disorder, score in sorted_disorders:
        print(f"- {disorder}: {score:.2f}")

    # Get top disorder
    top_disorder = sorted_disorders[0][0]
    print(f"\n*** Top Concern: {top_disorder}")

    # 7. Fetch recommendations
    rec_row = df_recs[df_recs['Disorder'].str.lower() == top_disorder.lower()]
    if not rec_row.empty:
      print("\n* Self-care:")
      for i, val in enumerate(rec_row['Reccomendations; Self'].dropna().values[:5], 1):
        print(f"  - Option {i}: {val}")

    print("\n* Professional Help:")
    for i, val in enumerate(rec_row['Reccomendation 2; Proffesional'].dropna().values[:3], 1):
        print(f"  - Option {i}: {val}")

    print("\n* Other Recommendations:")
    for i, val in enumerate(rec_row['Other Reccomendation'].dropna().values[:2], 1):
        print(f"  - Option {i}: {val}")
    else:
        print(" No recommendations found.")


In [65]:
predict_disorder_input("I am bored, I am restless, I cannot concentrate ")



 Predicted Disorders & Similarities:

- Bipolar disorder: 0.47
- Depression: 0.39
- Attention Deficit Hyperactivity Disorder(ADHD): 0.38

*** Top Concern: Bipolar disorder

* Self-care:
  - Option 1: Track moods using journals or apps to identify patterns and triggers
  - Option 2: Avoid triggers like poor quality sleep and overstimulation
  - Option 3: Stick to consistent routine
  - Option 4: Avoid over-comitting to a task
  - Option 5: Practice calming strategies like breathing exercises

* Professional Help:
  - Option 1: Visit your therapist for CBT(cognitive behavioural threapy)
  - Option 2: Monitor mood with clinical guided journals

* Other Recommendations:
 No recommendations found.


In [ ]:
# Save the model and embeddings
import pickle

nlp_bundle = {
    "model": model,
    "embeddings": disorder_embeddings,
    "signs_dict": signs_dict,
    "recommendations": df_recs
}

with open("model_nlp.pkl", "wb") as f:
    pickle.dump(nlp_bundle, f)
